In [1]:
from langchain_ollama import ChatOllama

from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from typing import Literal

In [2]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [3]:
from langgraph.prebuilt import ToolNode

@tool
def search(query: str):
    """Run web search on the question.
    
    Args:
        query (str): The query to search for.

    Returns:
        str: The answer to the question.
    """
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."




In [4]:
from scraping.db.documents import ArticleDocument
from scraping.entry import handler

@tool
def retrieve_medium_article(url: str):
    """Get the content of the Medium article.
    Args:
        url (str): The exact link of the article.

    Returns:
        str: The article content.
    """
    event = {
        "user": "Genereux Alahassa",
        "link": url,
    }
    handler(event, None)
    
    data = ArticleDocument.get()
    content = data.content['Content']
    return ' '.join(content.split())

Connection to database with uri: mongodb://127.0.0.1:27017/scrabble successful


In [5]:
retrieve_medium_article('https://medium.com/ai-in-plain-english/claude-3-5-sonnet-why-it-is-better-than-chatgpt-7709d7cbc237')

/var/folders/pb/_cl6j01j2vj04jsvwgpgtvvh0000gn/T/ipykernel_25573/2746252747.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieve_medium_article('https://medium.com/ai-in-plain-english/claude-3-5-sonnet-why-it-is-better-than-chatgpt-7709d7cbc237')


Starting scrapping Medium article: https://medium.com/ai-in-plain-english/claude-3-5-sonnet-why-it-is-better-than-chatgpt-7709d7cbc237
Successfully scraped and saved article: https://medium.com/ai-in-plain-english/claude-3-5-sonnet-why-it-is-better-than-chatgpt-7709d7cbc237


"Claude 3.5 Sonnet — Why it is BETTER than ChatGPT 😮 | by Andrew Best | Artificial Intelligence in Plain EnglishOpen in appSign upSign inWriteSign upSign inMember-only storyClaude 3.5 Sonnet — Why it is BETTER than ChatGPT 😮The “artifact” feature is a game changerAndrew Best·FollowPublished inArtificial Intelligence in Plain English·4 min read·Jun 26, 20241K22ListenShareFor a while, I was getting used to only using ChatGPT. (Stupid, I know)I’m on the paid plan of ChatGPT, and I felt like there was no point in using anything else.Wrong ❌It is kind of like using Google — I don’t even think about using other search engines. I just blindly go to Google every time (although I’m now starting to use Perplexity)If you don’t have a paid Medium account, you can read for free here.I saw something recently about how Claude 3.5 Sonnet is the best model, even better than ChatGPT (GPT-4o).I also kept seeing these 2 pics everywhere:Claude 3.5 Sonnet vs other Claude ModelsandClaude 3.5 Sonnet vs ChatGP

In [6]:
tools = [search, retrieve_medium_article]

In [7]:
llm = ChatOllama(model="llama3-groq-tool-use", temperature=0)
#model = ChatOllama(model="llama3.1", temperature=0)
llm_with_tools = llm.bind_tools(tools)


def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)

In [8]:
tool_node = ToolNode(tools=tools)

In [9]:
graph_builder.add_node("tools", tool_node)

In [10]:
def route_tools(
    state: State,
):
    """
    Use in the conditional_edge to route to the ToolNode if the last message
    has tool calls. Otherwise, route to the end.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return END


# The `tools_condition` function returns "tools" if the chatbot asks to use a tool, and "END" if
# it is fine directly responding. This conditional routing defines the main agent loop.
graph_builder.add_conditional_edges(
    "chatbot",
    route_tools,
    # The following dictionary lets you tell the graph to interpret the condition's outputs as a specific node
    # It defaults to the identity function, but if you
    # want to use a node named something else apart from "tools",
    # You can update the value of the dictionary to something else
    # e.g., "tools": "my_tools"
    {"tools": "tools", END: END},
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")
graph = graph_builder.compile()

In [14]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [("user", user_input)]}):
        for value in event.values():
            print("\n")
            print("Assistant:", value["messages"][-1].content)
                  

In [12]:
#stream_graph_updates("User: Ask me a question about a subject of your choice. The answer must be yes or no.")

In [13]:
stream_graph_updates("User: Using the function to retrieve Medium article, get the content of this Medium article https://medium.com/ai-in-plain-english/claude-3-5-sonnet-why-it-is-better-than-chatgpt-7709d7cbc237  and do a summary")

Assistant: 
Starting scrapping Medium article: https://medium.com/ai-in-plain-english/claude-3-5-sonnet-why-it-is-better-than-chatgpt-7709d7cbc237
Successfully scraped and saved article: https://medium.com/ai-in-plain-english/claude-3-5-sonnet-why-it-is-better-than-chatgpt-7709d7cbc237
Assistant: Claude 3.5 Sonnet — Why it is BETTER than ChatGPT 😮 | by Andrew Best | Artificial Intelligence in Plain EnglishOpen in appSign upSign inWriteSign upSign inMember-only storyClaude 3.5 Sonnet — Why it is BETTER than ChatGPT 😮The “artifact” feature is a game changerAndrew Best·FollowPublished inArtificial Intelligence in Plain English·4 min read·Jun 26, 20241K22ListenShareFor a while, I was getting used to only using ChatGPT. (Stupid, I know)I’m on the paid plan of ChatGPT, and I felt like there was no point in using anything else.Wrong ❌It is kind of like using Google — I don’t even think about using other search engines. I just blindly go to Google every time (although I’m now starting to use P

In [ ]:
while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

ModuleNotFoundError: No module named 'errors'